In [1]:
from importlib import reload # python 2.7 does not require this
from yahoofinancials import YahooFinancials
import numpy as np
import json as js
import datetime as dt

In [195]:
class cookFinancials(YahooFinancials):
    ticker = ''
    bshData = []
    ish = []
    cfsh = []
    summaryData = []
    def __init__(self, ticker):
        if isinstance(ticker, str):
            self.ticker = ticker.upper()
        else:
            self.ticker = [t.upper() for t in ticker]
        self._cache = {}
        
    def get_balanceSheetHistory(self):
        self.bshData = self.get_financial_stmts('annual', 'balance')['balanceSheetHistory']
        return self.bshData
    
    def get_incomeStatementHistory(self):
        self.ish = self.get_financial_stmts('annual', 'income')['incomeStatementHistory']
        return self.ish
    
    def get_cashflowStatementHistory(self):
        self.cfsh = self.get_financial_stmts('annual','cash')['cashflowStatementHistory']
        return self.cfsh
    
    def get_BV(self, numofYears=20):
        bv = []
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(min(np.size(self.bshData[self.ticker]), numofYears)):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            if not(self.bshData[self.ticker][i][date_key]):    
                break
            bv.append(self.bshData[self.ticker][i][date_key]['totalStockholderEquity'])
        return bv   
    
    def get_ROIC(self, numofYears=20):
        roic = []
        if not(self.cfsh):
            self.get_cashflowStatementHistory()
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(min(np.size(self.bshData[self.ticker]), numofYears)):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            if not(self.bshData[self.ticker][i][date_key]):    
                break
            equity = self.bshData[self.ticker][i][date_key]['totalStockholderEquity']
            if self.bshData[self.ticker][i][date_key].get('shortLongTermDebt') is None or not(self.bshData[self.ticker][i][date_key]['shortLongTermDebt']):
                debt_short = 0
            else:
                debt_short = self.bshData[self.ticker][i][date_key].get('shortLongTermDebt')
            if self.bshData[self.ticker][i][date_key].get('longTermDebt') is None or not(self.bshData[self.ticker][i][date_key]['longTermDebt']) :
                debt_long = 0
            else:
                debt_long = self.bshData[self.ticker][i][date_key]['longTermDebt']
            debt = debt_short + debt_long
            date_key = list(self.cfsh[self.ticker][i].keys())[0]
            if not(self.cfsh[self.ticker][i][date_key]):    
                break
            netincome = self.cfsh[self.ticker][i][date_key]['netIncome']
            roic_year = netincome/(equity + debt)
            roic.append(roic_year)
        return roic 
    
    def get_totalCashFromOperatingActivities(self, numofYears=20):
        totalCash = []
        if not(self.cfsh):
            self.get_cashflowStatementHistory()        
        for i in range(min(np.size(self.cfsh[self.ticker]), numofYears)):
            date_key = list(self.cfsh[self.ticker][i].keys())[0]
            if not(self.cfsh[self.ticker][i][date_key]):    
                break
            totalCash.append(self.cfsh[self.ticker][i][date_key]['totalCashFromOperatingActivities'])  
        return totalCash
    
    def get_pricetoSales(self):
        if not(self.summaryData):
            self.summaryData = self.get_summary_data()
        if not(self.summaryData[self.ticker]):
            return 'na'
        return self.summaryData[self.ticker]['priceToSalesTrailing12Months']
    
    def get_marketCap_B(self):
        if not(self.summaryData):
            self.summaryData = self.get_summary_data()
        if not(self.summaryData[self.ticker]):
            return 'na'
        return self.summaryData[self.ticker]['marketCap']/1000000000
    
    def get_CF_GR_median(self, totalCash):
        gr = []
        for v in range(np.size(totalCash)-1):
            gr.append((totalCash[v]-totalCash[v+1])/abs(totalCash[v+1]))
        #print(gr)
        return np.size(totalCash)-1, np.median(gr) 
    
    #use mean of each year    
    def get_BV_GR_median(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.median(gr)
    
    #use mean of each year    
    def get_ROIC_median(self, roic):
        return np.size(roic), np.median(roic)
    
    def get_BV_GR_max(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.max(gr)
    
    def growthRate(self, cur,init, years):
        if cur <=0 or init<=0:
            return -1
        return (cur/init)**(1/years)-1
    
    def get_BV_GR_mean(self, bv):
        gr = []
        BV_GR = self.growthRate(bv[0], bv[np.size(bv)-1], np.size(bv)-1)
        if BV_GR==-1:
            for v in range(np.size(bv)-1):
                gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
            BV_GR = np.mean(gr)
        return np.size(bv)-1, BV_GR
    
    def get_suggest_price(self, cEPS, growth, years, rRate, PE, safty):
        if not(cEPS) or not(growth) or not(PE):
            return 'NA'
        fEPS = cEPS*(1+growth)**years
        fPrice = fEPS*PE;
        stickerPrice = fPrice/(1+rRate)**years
        return stickerPrice, stickerPrice*safty
    
    def payBackTime(self, price, cEPS, growth):
        tmp = 0
        i = 0
        if cEPS < 0:
            return 0
        while(growth>0):
            i+=1
            tmp = tmp + cEPS*(1+growth)**i
            if (tmp>price):
                break
        return i
    
    def get_earningsperShare(self):
        eps = x.get_earnings_per_share()
        if not(eps):
            eps = x.get_key_statistics_data()[self.ticker]['trailingEps']
        print(eps)
        return eps
    
    def get_PE(self):
        #print(self._stock_summary_data('trailingPE'))
        #print(self._stock_summary_data('forwardPE'))
        if not(self._stock_summary_data('trailingPE')):
            return self._stock_summary_data('forwardPE')
        if not(self._stock_summary_data('forwardPE')):
            return self._stock_summary_data('trailingPE')
        return (self._stock_summary_data('trailingPE')+self._stock_summary_data('forwardPE'))/2
    
    def get_decision(self,suggestPrice, stockprice):
        #print('suggested price:', suggestPrice)
        #print('stock price:', stockprice)
        if isinstance(suggestPrice, str):
            return 'skip due to negative eps'
        elif suggestPrice>stockprice:
            return 'strong buy' 
        else:
            return 'do not buy'   
    def get_ma(self, date_from, data_to):
        data = self.get_historical_price_data(date_from,date_to, 'daily')
        tmp = 0
        if not(data[self.ticker]['prices']):
            return -1
        for i in range(len(data[self.ticker]['prices'])):
            #print(data[self.ticker]['prices'][i]['formatted_date'])
            tmp = tmp + data[self.ticker]['prices'][i]['close']
        return tmp/(i+1)
    def get_ma_50(self, date):
        date_from = str(date - dt.timedelta(days=50))
        date_to = str(date)
        return self.get_ma(date_from, date_to)
    def get_ma_200(self, date):
        date_from = str(date - dt.timedelta(days=200))
        date_to = str(date)
        return self.get_ma(date_from, date_to)
    def get_ma_150(self, date):
        date_from = str(date - dt.timedelta(days=150))
        date_to = str(date)
        return self.get_ma(date_from, date_to)
    def get_30day_trend_ma200(self):
        ###no need to look at everyday, just check last, mid, current
        current = self.get_ma_200((dt.date.today()))
        print(dt.date.today())
        print(current)
        mid = self.get_ma_200((dt.date.today()-dt.timedelta(days=15)))
        print(dt.date.today()-dt.timedelta(days=15))
        print(mid)
        last = self.get_ma_200((dt.date.today()-dt.timedelta(days=30)))
        print(dt.date.today()-dt.timedelta(days=30))
        print(last)
        if current - mid > 0 and mid -last > 0:
            return 1
        else:
            return -1
    def mv_strategy(self):
        currentPrice = self.get_current_price()
        price50 = self.get_ma_50(dt.date.today())
        price150 = self.get_ma_150(dt.date.today())
        price200 = self.get_ma_200(dt.date.today())
        if currentPrice > price50 and currentPrice > price150 and currentPrice > price200 and price150 > price200 and price50 > price150 and price50 > price200 and self.get_30day_trend_ma200() == 1:
            return 1
        else:
            return -1
    def        

In [560]:
class batch_process:
    def __init__(self, tickers, section):
        self.tickers = tickers
        self.jsfile = section
        with open(section, "w") as f:
            s = {"data":[]}
            js.dump(s, f, indent = 4)
    def batch_run(self):       
        for i in range(np.size(self.tickers)):
            try:
                print(self.tickers[i])
                x = cookFinancials(self.tickers[i])
                bv = x.get_BV(20)
                bv.insert(0, x.get_book_value())
                print(bv)
                bvgr = x.get_BV_GR_median(bv)
                print(bvgr)
                growth = bvgr[1]
                cEPS = x.get_earnings_per_share()
                print(cEPS)
                years = 3;
                rRate = 0.25;
                safty = 0.5
                PE = x.get_PE()
                price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
                print(price)
                stickerPrice = x.get_current_price()
                decision = x.get_decision(price[1],stickerPrice)
                print(decision)
                y2pb = 0
                roic = 0
                mcap = 0
                cashflow = 0
                priceSales = 0
                if decision == 'strong buy':
                    y2pb = x.payBackTime(stickerPrice, cEPS, growth)
                    roic = x.get_ROIC()
                    mcap = x.get_marketCap_B()
                    cashflow = (x.get_totalCashFromOperatingActivities())
                    priceSales = x.get_pricetoSales()               
                s = {
                    self.tickers[i]:{
                        "decision":decision,
                        "suggested price":price[1],
                        "stock price":x.get_current_price(),                     
                        "Payback years": y2pb,
                        "Book Value": bv,
                        "ROIC": roic,
                        "market cap (b)": mcap,
                        "cashflow": cashflow,
                        "priceSalesRatio":priceSales,
                        "PE": PE
                    }
                }
                print(s)
                with open(self.jsfile, "r") as f:
                    data = js.load(f)
                    cont = data['data']
                    cont.append(s)
                with open(self.jsfile, "w") as f:
                    js.dump(data, f, indent=4) 
                print('=====================================')
            except Exception:
                pass

In [196]:
x = cookFinancials('MGLN')
date_from = str(dt.date.today() - dt.timedelta(days=2))
date_to = str(dt.date.today())

In [189]:
x.get_ma_200(dt.date.today())

76.30277335730783

In [197]:
x.get_30day_trend_ma200()

2021-01-04
76.30277335730783
2020-12-20
76.02905376537426
2020-12-05
75.79968319663519


1

In [198]:
x.mv_strategy()

2021-01-04
76.30277335730783
2020-12-20
76.02905376537426
2020-12-05
75.79968319663519


1

In [546]:
bv = x.get_BV(20)
bv.insert(0, x.get_book_value())
print(bv)
bvgr = x.get_BV_GR_median(bv)
print(bvgr)
growth = bvgr[1]
cEPS = x.get_earnings_per_share()
years = 3;
rRate = 0.25;
safty = 0.5
PE = x.get_PE()
price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
                

[1499918000, 833943000, -7439000, -26671000, -28838000]
(4, 0.7598342893514913)


In [547]:
print(price)
stickerPrice = x.get_current_price()
decision = x.get_decision(price[1],stickerPrice)
print(decision)
y2pb = 0
roic = 0
mcap = 0
priceSales = 0

(775.3858606329167, 387.69293031645833)
strong buy


In [562]:
from get_all_tickers import get_tickers as gt
from get_all_tickers.get_tickers import Region

In [563]:
tickers = gt.get_tickers()



In [591]:
filtered_by_sector = gt.get_tickers_filtered(mktcap_min=1e3, sectors=gt.SectorConstants.ENERGY)

In [592]:
filtered_by_sector

['AR',
 'BKR',
 'BBL',
 'BSM',
 'BE',
 'BPMP',
 'BP',
 'BC',
 'BWXT',
 'COG',
 'WHD',
 'CRC',
 'CNQ',
 'CVE',
 'CVX',
 'SNP',
 'XEC',
 'CEO',
 'CNX',
 'CRK',
 'CXO',
 'COP',
 'CLR',
 'CLB',
 'CPG',
 'CMI',
 'CVI',
 'DKL',
 'DK',
 'DEN',
 'DVN',
 'DRQ',
 'EC',
 'EMR',
 'ENB',
 'E',
 'EOG',
 'EQT',
 'EQNR',
 'XOM',
 'GE',
 'EAF',
 'HAL',
 'HP',
 'HES',
 'HEP',
 'HFC',
 'LBRT',
 'MIC',
 'MMP',
 'MGY',
 'MRO',
 'MPC',
 'MTDR',
 'MPLX',
 'MUR',
 'NOV',
 'NS',
 'OXY',
 'OTIS',
 'OVV',
 'PE',
 'PBA',
 'PTR',
 'PBR',
 'PSX',
 'PSXP',
 'PXD',
 'RRC',
 'SSL',
 'SLB',
 'SHLX',
 'SWN',
 'SU',
 'SUN',
 'FTI',
 'TPL',
 'TOT',
 'RIG',
 'VLO',
 'HCC',
 'INT',
 'WPX',
 'YPF',
 'APA',
 'BLDP',
 'BROG',
 'FANG',
 'HPK',
 'MGEE',
 'PDCE',
 'PTON',
 'PAA',
 'PAGP',
 'PLUG',
 'TPIC',
 'VNOM',
 'WWD',
 'IMO']

In [593]:
y = batch_process(filtered_by_sector, 'energy_1b.json')

In [594]:
y.batch_run()


AR
[5668127000, 6970743000, 7665808000, 8149181000, 6262625000]
(4, -0.07499316926075862)
None
NA
skip due to negative eps
{'AR': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 5.75, 'Payback years': 0, 'Book Value': [5668127000, 6970743000, 7665808000, 8149181000, 6262625000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': -52.272728}}
BKR
[11582000000, 21929000000, 17465000000, 14277000000, 14688000000]
(4, 0.09765709227236456)
None
NA
skip due to negative eps
{'BKR': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 21.01, 'Payback years': 0, 'Book Value': [11582000000, 21929000000, 17465000000, 14277000000, 14688000000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': None}}
BBL
[47936000000, 47936000000, 47240000000, 55592000000, 57258000000]
(4, -0.014548185406406091)
3.140000223690808
(30.13465601863955, 15.067328009319775)
do not buy
{'BBL': {'decision': 'do not

[4270176000, 4160546000, 4329958000, 3899899000, 3798395000]
(4, 0.026536386400673545)
None
NA
skip due to negative eps
{'CNX': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 11.46, 'Payback years': 0, 'Book Value': [4270176000, 4160546000, 4329958000, 3899899000, 3798395000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': 11.814433}}
CRK
[1187680000, 1143022000, 569571000, -369272000]
(3, 1.0068121445789902)
None
NA
skip due to negative eps
{'CRK': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 4.49, 'Payback years': 0, 'Book Value': [1187680000, 1143022000, 569571000, -369272000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': 6.507246}}
CXO
[7787000000, 17782000000, 18768000000, 8915000000, 7623000000]
(4, 0.05847542334696485)
None
NA
skip due to negative eps
{'CXO': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 57.46, 'Payback y

None
NA
skip due to negative eps
{'EOG': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 49.6, 'Payback years': 0, 'Book Value': [20148326000, 21640716000, 19364188000, 16283273000, 13981581000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': 21.659388}}
EQT
[8845809000, 9803588000, 10958229000, 13319618000, 5860281000]
(4, -0.1015321321586743)
None
NA
skip due to negative eps
{'EQT': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 13.91, 'Payback years': 0, 'Book Value': [8845809000, 9803588000, 10958229000, 13319618000, 5860281000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': 28.979168}}
EQNR
[34084000000, 41139000000, 42970000000, 39861000000, 35072000000]
(4, 0.017692456092928514)
None
NA
skip due to negative eps
{'EQNR': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 16.29, 'Payback years': 0, 'Book Value': [34084000000, 411390

[4343436000, 5467460000, 4829299000, 4620191000, 4916679000]
(4, -0.007521446030538292)
None
NA
skip due to negative eps
{'MUR': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 12.16, 'Payback years': 0, 'Book Value': [4343436000, 5467460000, 4829299000, 4620191000, 4916679000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': -11.580953}}
NOV
[5465000000, 7778000000, 13819000000, 14094000000, 13940000000]
(4, -0.15844453340377102)
None
NA
skip due to negative eps
{'NOV': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 13.5, 'Payback years': 0, 'Book Value': [5465000000, 7778000000, 13819000000, 14094000000, 13940000000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': -51.92308}}
NS
[535760000, 1019909000, 1501430000, 1685660000, 1361465000]
(4, -0.21500038038358377)
None
NA
skip due to negative eps
{'NS': {'decision': 'skip due to negative eps', 'suggested price': 'A',

[36344000000, 42042000000, 44005000000, 45383000000, 44630000000]
(4, -0.03748617990891963)
2.26799986153264
(8.330122702656874, 4.165061351328437)
do not buy
{'SU': {'decision': 'do not buy', 'suggested price': 4.165061351328437, 'stock price': 17.1, 'Payback years': 0, 'Book Value': [36344000000, 42042000000, 44005000000, 45383000000, 44630000000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': 8.0448415}}
SUN
[635000000, 758000000, 784000000, 1947000000, 2196000000]
(4, -0.13782855371483774)
1.5900000678726411
(6.765788493449277, 3.3828942467246383)
do not buy
{'SUN': {'decision': 'do not buy', 'suggested price': 3.3828942467246383, 'stock price': 28.58, 'Payback years': 0, 'Book Value': [635000000, 758000000, 784000000, 1947000000, 2196000000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': 12.9679225}}
FTI
[4189100000, 7659300000, 10357600000, 13387900000, 5055800000]
(4, -0.2434300984785187)
None
NA
skip due to negative eps


[1801000000, 1678000000, -538600000, -315600000, -281600000]
(4, -0.023718543449994585)
None
NA
skip due to negative eps
{'PTON': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 162.72, 'Payback years': 0, 'Book Value': [1801000000, 1678000000, -538600000, -315600000, -281600000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': -167.75258}}
PAA
[7414000000, 10770000000, 9710000000, 8665000000, 7251000000]
(4, 0.11488296192585558)
None
NA
skip due to negative eps
{'PAA': {'decision': 'skip due to negative eps', 'suggested price': 'A', 'stock price': 8.85, 'Payback years': 0, 'Book Value': [7414000000, 10770000000, 9710000000, 8665000000, 7251000000], 'ROIC': 0, 'market cap (b)': 0, 'cashflow': 0, 'priceSalesRatio': 0, 'PE': 6.9685044}}
PAGP
[1472000000, 2155000000, 1846000000, 1695000000, 1737000000]
(4, 0.032452962844085775)
None
NA
skip due to negative eps
{'PAGP': {'decision': 'skip due to negative eps', 'suggested price': 'A

In [ ]:
t = 'VCEL'

In [165]:
t[1]

'C'